In [1]:
import numpy as np
import pandas as pd
from skimage.io import imread, imshow, imread_collection, concatenate_images
import os
from os.path import join
import glob
import cv2
import re
%matplotlib inline
import matplotlib.pyplot as plt
from skimage.transform import resize

TRAIN_PATH = '/home/alvin/cei/notebook_home/kaggleDSB2018/data/stage1_train/'
TEST_PATH = '/home/alvin/cei/notebook_home/kaggleDSB2018/data/stage1_test/'
IMG_WIDTH = 32
IMG_HEIGHT = 32
IMG_CHANNELS = 3

train_ids = os.listdir(TRAIN_PATH)
test_ids = os.listdir(TEST_PATH)

train_ids = list(filter(lambda x: re.search('.png',x), train_ids))
test_ids = list(filter(lambda x: re.search('.png',x), test_ids))

train_ids = [re.sub('.png','',x) for x in train_ids]
test_ids = [re.sub('.png','',x) for x in test_ids]

In [2]:
train_image_paths = [glob.glob(join(TRAIN_PATH, train_id, "images", "*"))[0] for train_id in train_ids]
test_image_paths = [glob.glob(join(TEST_PATH, test_id, "images", "*"))[0] for test_id in test_ids]

from tqdm import tqdm

In [17]:
def get_image_finfo(image_paths):
    
    cei_csv = pd.read_csv('./stage1_train_fix_v4_external.csv')
    
    # complete img ,rgb mode
    full_img_list = []
    # just grey mode
    img_list = []
    
    # average value of gray pixels per image
    average_list = []
    
    # max Contour area value per image
    max_cnt_area = []
    
    # mean Contour area value per image
    average_cnt_area = []
    
    # how many Contour areas per image
    num_cnt = []
    
    #  width per image
    wid_list = []
    
    #  length per image
    len_list = []
    
    #  red per image
    r=[]
    
    #  green
    g=[]
    
    #  blue
    b=[]
    
    #  ground truth
    category = []
    sub_category = []
    
    #  source
    source = []
    
    for case in tqdm(image_paths, total=len(image_paths)): 
        img = imread(case)[:,:,:IMG_CHANNELS]
        full_img_list.append(img)  
        r.append(np.average(img[:,:,0]))
        g.append(np.average(img[:,:,1]))
        b.append(np.average(img[:,:,2]))
        
        img = cv2.imread(case,cv2.IMREAD_GRAYSCALE)
        
        # in some cases, image background is bright and cell darker, there needs a inverse of pixel value
        if np.average(img) > 125:
            img = 255 - img   
        img_list.append(img)

        lenth = img.shape[0]
        len_list.append(lenth)
        width = img.shape[1]
        wid_list.append(width)
        average_list.append(np.average(img))
        
        # use opencv to find contour and get some stactistic data
        img = cv2.GaussianBlur(img, (3, 3), 1)
        ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU)

        _, cnts, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
        max_cnt_area.append(cv2.contourArea(cnts[0])/lenth/width)

        av = 0
        for i in cnts:
            av = av + cv2.contourArea(i)
        av = av/len(cnts)
        
        # since different pic has different size, we'd better normalise it 
        average_cnt_area.append(av/lenth/width)
        num_cnt.append(len(cnts))
        
        # get ground truth for category and sub_category from kaggle
        uid = re.sub('.*/|.png','',case)
        print(uid)
        category.append(str(cei_csv[cei_csv['image_id'==uid]]['major_category']))
        sub_category.append(cei_csv[cei_csv['image_id'==uid]]['sub_category'])
        
        # get source information from kaggle
        
        source.append(cei_cst[cei_csv['image_id'==uid]]['source'])
    df = pd.DataFrame({'img':full_img_list,'max_area':max_cnt_area,'average_area':average_cnt_area,
                       'num_cnt':num_cnt,'average':average_list,'wid':wid_list,'len':len_list,
                       'r':r,'g':g,'b':b, 'source':source, 'category':category, 'sub_category':sub_category
                      }) 
    return df

In [18]:
df = get_image_finfo(train_image_paths)

  0%|          | 0/670 [00:00<?, ?it/s]

dd54adb80393de7769b9853c0aa2ee9b240905d0e99c59d4ccd99401f327aa05


KeyError: False

In [ ]:
len(df)

In [ ]:
# divide into 3 categories according to characters belong to form
FDIV = 3
# divide into 3 categories according to characters belong to color
CDIV = 3

In [ ]:
from sklearn.cluster import KMeans 

# train seperately
input_x = np.array(df[['max_area','average_area','num_cnt','average','wid','len']])
  
fkmeans = KMeans(n_clusters = FDIV).fit(input_x) 

df['flabel'] = fkmeans.labels_

input_c = np.array(df[['r','g','b']])
  
ckmeans = KMeans(n_clusters = CDIV).fit(input_c) 

df['clabel'] = ckmeans.labels_

# and then make an combination
df['cflabel'] = FDIV *df['flabel']

df['cflabel'] = df['cflabel'] + df['clabel']

df['cflabel'].hist()

In [ ]:
df['cflabel'].as_matrix()

In [ ]:
gg = df['cflabel'].as_matrix()
print('class 0 =>' + str(len(gg[gg==0])))
print('class 1 =>' + str(len(gg[gg==1])))
print('class 2 =>' + str(len(gg[gg==2])))
print('class 3 =>' + str(len(gg[gg==3])))
print('class 4 =>' + str(len(gg[gg==4])))
print('class 5 =>' + str(len(gg[gg==5])))
print('class 6 =>' + str(len(gg[gg==6])))
print('class 7 =>' + str(len(gg[gg==7])))
print('class 8 =>' + str(len(gg[gg==8])))

In [ ]:
for t in range(FDIV*CDIV):
    print('for type=>'+str(t))
    fig,ax= plt.subplots(4,10,figsize=(64,10))
    
    n=0
    for i in range(4):
        for j in range(10):
            if n < len(df[df['cflabel']==t].index):
                sn = df[df['cflabel']==t].index[n]
                ax[i,j].imshow(df.img[sn])
                n = n+1
    plt.show()

In [ ]:
CUST_CATEGORY = 4
akmeans = KMeans(n_clusters = CUST_CATEGORY).fit(input_c) 

df['alabel'] = akmeans.labels_
df['alabel'].hist()

In [ ]:
gg = df['alabel'].as_matrix()
gg

In [ ]:
for t in range(CUST_CATEGORY):
    print('for type=>'+str(t))
    fig,ax= plt.subplots(4,10,figsize=(64,10))
    
    n=0
    for i in range(4):
        for j in range(10):
            if n < len(df[df['alabel']==t].index):
                sn = df[df['alabel']==t].index[n]
                ax[i,j].imshow(df.img[sn])
                n = n+1
    plt.show()

In [12]:
df[df['alabel']==2]

NameError: name 'df' is not defined

In [19]:
cei_csv = pd.read_csv('./stage1_train_fix_v4_external.csv')

In [25]:
uid = 'dd54adb80393de7769b9853c0aa2ee9b240905d0e99c59d4ccd99401f327aa05'
cei_csv['image_id'==uid].index

KeyError: False